<a href="https://colab.research.google.com/github/leothey-del/heartmula/blob/main/HeartMuLa_by_AIQUEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎵 HeartMuLa 3B Music Generator - Free Tier Optimized
### Created by AIQUEST (@AIQuestAcademy)

---

## 📖 About This Notebook

This notebook allows you to run **HeartMuLa 3B**, an open-source AI music generation model, on **Google Colab's FREE tier** using BF16 optimization and lazy loading techniques.

### ✨ Key Features
- 🆓 **Free Tier Compatible** - Runs on T4 GPU (15GB VRAM)
- 🎼 **Professional Quality** - Generate music with lyrics and style control
- ⚡ **BF16 Optimized** - 50% memory reduction vs full precision
- 🎚️ **Advanced Controls** - Temperature, Top-K, CFG Scale adjustments
- 🌍 **Multilingual Support** - Generate music in multiple languages
- ⏱️ **Time Estimate** - ~5-7 minutes per minute of audio

### 📊 Technical Specs
- **Model:** HeartMuLa-oss-3B (BF16)
- **Components:** HeartCodec, HeartMuLaGen, HeartTranscriptor
- **Framework:** PyTorch + Gradio
- **GPU Required:** NVIDIA T4 or better (15GB+ VRAM)

### 🎬 Support AIQUEST
If you find this notebook useful, please support my work:
- 🔴 **YouTube:** [@AIQuestAcademy](https://youtube.com/@AIQuestAcademy)
- 🐦 **X/Twitter:** [@AIQuestAcademy](https://twitter.com/AIQuestAcademy)

I spend hours optimizing and testing these models so you don't have to!

### ⚠️ Important Notes
- **Generation Time:** Expect 5-7 minutes per minute of audio
- **Colab Timeout:** Free tier sessions timeout after 90 minutes of inactivity
- **First Run:** Initial model loading may take extra time

### 📚 Credits
- **Original Model:** [HeartMuLa Team](https://github.com/HeartMuLa/heartlib)
- **Research Paper:** [arXiv:2601.10547](https://arxiv.org/abs/2601.10547)
- **BF16 Checkpoints:** [benjiaiplayground](https://huggingface.co/benjiaiplayground)
- **Notebook Optimization:** AIQUEST (@AIQuestAcademy)

---

## 🚀 Quick Start Guide

1. **Run Cell 1:** Install dependencies and setup environment (~3-5 minutes)
2. **Run Cell 2:** Download BF16 models (~5-10 minutes, one-time only)
3. **Run Cell 3:** Launch Gradio interface and start generating music!

**Ready? Let's make some music! 🎵**

---


In [7]:
# @title Setup Environment
import os

print("🔧 Setting up environment...")
print("="*60)

# Step 1: Install system dependencies
print("\n[1/5] 📦 Installing FFmpeg...")
!sudo apt-get update -qq && sudo apt-get install -y ffmpeg > /dev/null 2>&1
print("✓ FFmpeg installed")

# Step 2: Clone repository (if not already cloned)
print("\n[2/5] 📥 Cloning HeartMuLa repository...")
if not os.path.exists("/content/heartlib"):
    !git clone https://github.com/HeartMuLa/heartlib.git /content/heartlib -q
    print("✓ Repository cloned")
else:
    print("✓ Repository already exists")

# Step 3: Change to heartlib directory BEFORE installing
%cd /content/heartlib

# Step 4: Install Python packages in correct order
print("\n[3/5] 📦 Installing huggingface_hub...")
!pip install -q "huggingface_hub>=1.3.0,<2.0"
print("✓ huggingface_hub installed")

print("\n[4/5] 📦 Installing HeartMuLa package...")
!pip install -q -e .
print("✓ HeartMuLa package installed")

print("\n[5/5] 📦 Installing additional dependencies...")
!pip install -q flash-attn --no-build-isolation
!pip install -q accelerate
print("✓ Additional dependencies installed")

print("\n" + "="*60)
print("✅ Environment setup complete!")
print("="*60)

🔧 Setting up environment...

[1/5] 📦 Installing FFmpeg...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
✓ FFmpeg installed

[2/5] 📥 Cloning HeartMuLa repository...
✓ Repository already exists
/content/heartlib

[3/5] 📦 Installing huggingface_hub...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.0 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.5.0 which is incompatible.
✓ huggingface_hub installed

[4/5] 📦 Installing HeartMuLa package...
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for heartlib (pypro

In [8]:
# @title Download Models (BF16 Optimized)
import os
import shutil

# Ensure we're in the heartlib directory
if os.path.basename(os.getcwd()) != "heartlib":
    %cd /content/heartlib

os.makedirs('./ckpt', exist_ok=True)

print("="*70)
print("📥 DOWNLOADING HEARTMULA MODELS (BF16 OPTIMIZED)")
print("="*70)

# Model 1: HeartMuLa-oss-3B
print("\n[1/3] 📦 HeartMuLa-oss-3B (Main Generation Model)")
!huggingface-cli download --local-dir './ckpt/HeartMuLa-oss-3B' 'benjiaiplayground/HeartMuLa-oss-3B-bf16' --quiet
print("✓ Downloaded")

# Model 2: HeartCodec-oss
print("\n[2/3] 📦 HeartCodec-oss (Audio Codec)")
!huggingface-cli download --local-dir './ckpt/HeartCodec-oss' 'benjiaiplayground/HeartCodec-oss-bf16' --quiet
print("✓ Downloaded")

# Model 3: HeartMuLaGen (required for gen_config.json and tokenizer.json)
print("\n[3/3] 📦 HeartMuLaGen (Config & Tokenizer)")
!huggingface-cli download --local-dir './ckpt' 'HeartMuLa/HeartMuLaGen' --quiet
print("✓ Downloaded")

# Fix HeartCodec file naming
print("\n" + "="*70)
print("⚙️  CONFIGURING MODEL FILES...")
codec_file = './ckpt/HeartCodec-oss/HeartCodec-oss-bf16.safetensors'
if os.path.exists(codec_file):
    shutil.move(codec_file, './ckpt/HeartCodec-oss/model.safetensors')
    print("✓ HeartCodec checkpoint configured")
else:
    print("ℹ️ HeartCodec already configured or using different naming")

print("\n" + "="*70)
print("✅ ALL MODELS DOWNLOADED SUCCESSFULLY!")
print("="*70)

# Verify structure
print("\n📁 Checkpoint structure:")
!ls -la ./ckpt/

📥 DOWNLOADING HEARTMULA MODELS (BF16 OPTIMIZED)

[1/3] 📦 HeartMuLa-oss-3B (Main Generation Model)
⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
/content/heartlib/ckpt/HeartMuLa-oss-3B
✓ Downloaded

[2/3] 📦 HeartCodec-oss (Audio Codec)
⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
/content/heartlib/ckpt/HeartCodec-oss
✓ Downloaded

[3/3] 📦 HeartMuLaGen (Config & Tokenizer)
⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
/content/heartlib/ckpt
✓ Downloaded

⚙️  CONFIGURING MODEL FILES...
✓ HeartCodec checkpoint configured

✅ ALL MODELS DOWNLOADED SUCCESSFULLY!

📁 Checkpoint structure:
total 8908
drwxr-xr-x 5 root root    4096 Feb 28 08:52 .
drwxr-xr-x 8 root root    4096 Feb 28 07:26 ..
drwxr-xr-x 3 root root    4096 Feb 28 07:26 .cache
-rw-r--r-- 1 root root     101 Feb 28 07:26 gen_config.json
-rw-r--r-- 1 root root    1519 Feb 28 07:26 .gitattributes
drwxr-xr-x 3 root root    4096

In [9]:
# @title Launch HeartMuLa: Unique Seed Edition
import gradio as gr
import os
import subprocess
import time
from datetime import datetime
import torch
import numpy as np
import random

# Ensure we're in the heartlib directory
if os.path.basename(os.getcwd()) != "heartlib":
    if os.path.exists("/content/heartlib"):
        os.chdir("/content/heartlib")

def generate_music(lyrics, tags, duration, temperature, topk, cfg_scale, seed, progress=gr.Progress()):
    """Music generation with manual seed injection to bypass script limitations"""
    try:
        start_time = time.time()
        progress(0, desc="🚀 Initializing HeartMuLa 3B...")

        # 1. HANDLE SEED MANUALLY
        # If seed is -1, pick a random one. Otherwise use your number.
        actual_seed = int(seed) if seed != -1 else random.randint(1, 999999)

        # This locks the 'randomness' of the GPU and CPU globally
        torch.manual_seed(actual_seed)
        torch.cuda.manual_seed_all(actual_seed)
        np.random.seed(actual_seed)
        random.seed(actual_seed)
        torch.backends.cudnn.deterministic = True

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = f"./assets/music_{timestamp}.mp3"
        os.makedirs("./assets", exist_ok=True)

        # 2. CREATE TEMP FILES
        lyrics_file = f"./assets/lyrics_{timestamp}.txt"
        with open(lyrics_file, "w", encoding="utf-8") as f:
            f.write(lyrics)

        tags_file = f"./assets/tags_{timestamp}.txt"
        with open(tags_file, "w", encoding="utf-8") as f:
            f.write(tags)

        duration_ms = int(duration * 1000)

        # 3. BUILD COMMAND (Notice: --seed is REMOVED to prevent the error)
        cmd = [
            "python", "./examples/run_music_generation.py",
            f"--model_path=./ckpt",
            f"--version=3B",
            f"--lazy_load=true",
            f"--lyrics={lyrics_file}",
            f"--tags={tags_file}",
            f"--max_audio_length_ms={duration_ms}",
            f"--temperature={temperature}",
            f"--topk={int(topk)}",
            f"--cfg_scale={cfg_scale}",
            f"--save_path={output_path}"
        ]

        progress(0.2, desc=f"🧬 Melody Locked to Seed: {actual_seed}")
        progress(0.4, desc="🎼 Generating tokens (this takes time)...")

        # 4. RUN GENERATION
        result = subprocess.run(cmd, capture_output=True, text=True)
        elapsed_time = time.time() - start_time

        # Cleanup temp files
        try:
            os.remove(lyrics_file)
            os.remove(tags_file)
        except:
            pass

        if result.returncode == 0 and os.path.exists(output_path):
            progress(1.0, desc="✅ Generation complete!")
            stats = f"""
### ✅ Track Generated Successfully!
* **Locked Seed:** `{actual_seed}` (Proof of uniqueness)
* **Duration:** {duration}s
* **Process Time:** {elapsed_time/60:.1f} minutes
* **YouTube Note:** Use this seed in your description for copyright clarity.
            """
            return output_path, stats
        else:
            error_log = result.stderr if result.stderr else result.stdout
            return None, f"❌ Script Error:\n```\n{error_log[-1000:]}\n```"

    except Exception as e:
        return None, f"❌ System Error: {str(e)}"

# Interface Styling
custom_css = """
.gradio-container { background-color: #0b0b0b; color: #00ffcc; }
.generate-btn { background: linear-gradient(135deg, #00ffcc 0%, #0088ff 100%) !important; color: black !important; font-weight: bold !important; }
"""

with gr.Blocks(theme=gr.themes.Soft(), css=custom_css) as demo:
    gr.Markdown("# 🎵 HeartMuLa 3B: YouTube Producer UI")

    with gr.Row():
        with gr.Column(scale=1):
            lyrics_in = gr.Textbox(label="Lyrics / Structure", lines=10,
                                  value="[Intro]\n(Melodic Vicetone synth)\n\n[Chorus]\nWe are the fire, we are the light!\nRising up through the neon night!")
            tags_in = gr.Textbox(label="Style Tags",
                                value="progressive house, vicetone style, 128 bpm, sidechained synth, festival energy, high quality")

            with gr.Row():
                dur_slider = gr.Slider(30, 300, value=90, label="Duration (sec)")
                seed_in = gr.Number(label="Manual Seed", value=580160, precision=0)

            with gr.Accordion("Expert Controls", open=False):
                temp_slider = gr.Slider(0.7, 1.3, value=1.0, label="Temperature")
                cfg_slider = gr.Slider(1.0, 5.0, value=2.0, label="CFG (Style Adherence)")
                topk_slider = gr.Number(value=50, label="Top-K")

            gen_btn = gr.Button("🔥 GENERATE UNIQUE TRACK", variant="primary", elem_classes="generate-btn")

        with gr.Column(scale=1):
            gr.Markdown("### 🎧 Your Generated Audio")
            out_audio = gr.Audio(label="Download MP3", type="filepath")
            out_stats = gr.Markdown("Ready to produce.")

    gen_btn.click(
        fn=generate_music,
        inputs=[lyrics_in, tags_in, dur_slider, temp_slider, topk_slider, cfg_slider, seed_in],
        outputs=[out_audio, out_stats]
    )

demo.launch(share=True, debug=False)

/tmp/ipython-input-279/4159225311.py:100: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), css=custom_css) as demo:
/tmp/ipython-input-279/4159225311.py:100: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), css=custom_css) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0c6acafa6e5f6b0f23.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
